In this code all the features are loaded into a single dataframe containing:
NHANES features + 
Physical activity features (see code 5_physical_activity_feature_engineering.ipynb) + 
target binary (see code 4_building_target_variable.ipynb)

The NHANES dataset is stored in numerous .xpt files. These files are located in 5 different folders:
'Demographics', 'Dietary', 'Examination', 'Laboratory', 'Questionnaire'

This folder structure matches the one found at NHANES website: 
https://wwwn.cdc.gov/nchs/nhanes/continuousnhanes/default.aspx?BeginYear=2013

A detaile explanation of each variable can be found here:
- Demographics: https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Demographics&CycleBeginYear=2013
- Dietary: https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Dietary&CycleBeginYear=2013
- Examination: https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Examination&CycleBeginYear=2013
- Laboratory: https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Laboratory&CycleBeginYear=2013
- Questionnaire: https://wwwn.cdc.gov/nchs/nhanes/search/datapage.aspx?Component=Questionnaire&CycleBeginYear=2013

In [1]:
import pandas as pd
from os import getcwd, listdir, mkdir
from os.path import join, exists

In [2]:
data_location = join(getcwd(), "data")
xpt_location = join(data_location, "xpt") 

In [3]:
for folder in listdir(xpt_location):
    print(folder)
    print(listdir(join(xpt_location, folder)))

Demographics
['demo_h.xpt']
Dietary
['dr1iff_h.xpt', 'dr1tot_h.xpt', 'dr2iff_h.xpt', 'dr2tot_h.xpt', 'drxfcd_h.xpt', 'ds1ids_h.xpt', 'ds1tot_h.xpt', 'ds2ids_h.xpt', 'ds2tot_h.xpt', 'dsbi.xpt', 'dsii.xpt', 'dspi.xpt', 'dsqids_h.xpt', 'dsqtot_h.xpt']
Examination
['bmx_h.xpt', 'bpx_h.xpt', 'csx_h.xpt', 'dxxaac_h.xpt', 'dxxag_h.xpt', 'dxxfem_h.xpt', 'dxxfrx_h.xpt', 'dxxl1_h.xpt', 'dxxl2_h.xpt', 'dxxl3_h.xpt', 'dxxl4_h.xpt', 'dxxspn_h.xpt', 'dxxt10_h.xpt', 'dxxt11_h.xpt', 'dxxt12_h.xpt', 'dxxt4_h.xpt', 'dxxt5_h.xpt', 'dxxt6_h.xpt', 'dxxt7_h.xpt', 'dxxt8_h.xpt', 'dxxt9_h.xpt', 'dxxvfa_h.xpt', 'dxx_h.xpt', 'flxcln_h.xpt', 'mgx_h.xpt', 'ohxden_h.xpt', 'ohxper_h.xpt', 'ohxref_h.xpt', 'paxday_h.xpt', 'paxhd_h.xpt', 'paxhr_h.xpt', 'paxmin_h.xpt']
Laboratory
['aas_h.xpt', 'aa_h.xpt', 'alb_cr_h.xpt', 'alds_h.xpt', 'ald_h.xpt', 'amdgds_h.xpt', 'amdgyd_h.xpt', 'apob_h.xpt', 'bfrpol_h.xpt', 'biopro_h.xpt', 'cafe_h.xpt', 'cbc_h.xpt', 'chlmda_h.xpt', 'cot_h.xpt', 'cusezn_h.xpt', 'deet_h.xpt', 'ephpp_h.x

1. Each XPR file is loaded into a dataframe. Then each dataframe is appended to a list.

In [8]:
dataframes_list = []

# Iterate over each folder in the xpt_location directory
for folder in listdir(xpt_location):    
    # Iterate over each file in the current folder
    for name_xpt in listdir(join(xpt_location, folder)):
        # Check if the file is not one of the DXX files (DXX files are where our target variables are calculated from), nor one of the PAX files (physical activity time series), which are processed separately and merged below (pax.csv and target.csv)
        if not name_xpt.lower().startswith('pax') and not name_xpt.lower().startswith('dxx'):
            # Read the xpt file as a pandas dataframe
            df = pd.read_sas(join(xpt_location, folder, name_xpt))
            # Check if 'SEQN' column exists in the dataframe
            if 'SEQN' in df.columns:
                # Each dataframe should have a single row for each SEQN value. Each SEQN value represents a subject.
                if not df['SEQN'].duplicated().any():
                    # Append the dataframe to the dataframes list
                    dataframes_list.append(df)                    
                #else:
                    # Some dataframes have multiple entries for the same SEQN. For example some nutrition or medication dataframes, since they register multiple nutrients or medications for each subject. To include these files in the final dataframe, we would need to preprocess and aggregate the data for each subject.
                    #print("There are duplicated SEQN values in the dataframe: " + name_xpt)
            #else:
                # Some dataframes do not have a SEQN column. That is, rather than containing data for each subject, they provide other type of information (e.g. food codes). We do not need these dataframes, so they are not appended them to the dataframes list.
                #print(f"The dataframe from file {name_xpt} does not have a 'SEQN' column.")



2. Merging all dataframes to create the full dataframe with all of the NHANES variables

In [9]:
# Start with the first dataframe
full_df = dataframes_list[0]

# Loop through the rest and merge based on SEQN
for df in dataframes_list[1:]:
    full_df = full_df.merge(df, on='SEQN', how='outer')

c:\Users\Horacio\OneDrive - UNIVERSIDAD DE SEVILLA\Doctorado\1 TRABAJOS ABIERTOS\26 Estudio Proyecto Samsung NHANES\nhanes_analysis\.environment\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'WTDR2D_x', 'DRDINT_x', 'WTDRD1_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(
c:\Users\Horacio\OneDrive - UNIVERSIDAD DE SEVILLA\Doctorado\1 TRABAJOS ABIERTOS\26 Estudio Proyecto Samsung NHANES\nhanes_analysis\.environment\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'URXUCR_x'} in the result is deprecated and will raise a MergeError in a future version.
  return merge(
c:\Users\Horacio\OneDrive - UNIVERSIDAD DE SEVILLA\Doctorado\1 TRABAJOS ABIERTOS\26 Estudio Proyecto Samsung NHANES\nhanes_analysis\.environment\lib\site-packages\pandas\core\frame.py:9190: FutureWarning: Passing 'suffixes' which cause duplicate columns {'

IMPORTANT: The previous cells produces warnings since there are column name duplicates. That is, there are columns (other than SEQN) in the DataFrames that have the same name but are from different files, the merge operation will append suffixes (_x, _y, etc.) to make the names unique. This happens for example in nutrition data, where the same variables where measured in different days, and each day was stored in a different dataframe. Be aware of this if you expect or see columns with these appended names in the merged DataFrame.

3. Including the Physical Activity features in the full dataframe.

In [10]:
pax_df = pd.read_csv('pax.csv')
pax_df = pax_df.drop(columns=['Unnamed: 0']) #This column is not needed.


In [11]:
full_df = full_df.merge(pax_df, on='SEQN', how='outer')

4. Including the target variable at the end of the full dataframe. (TARGET_BINARY)

In [12]:
target_df = pd.read_csv('target.csv')

In [13]:
full_df = full_df.merge(target_df, on='SEQN', how='outer')

5. Fixing read_sas bug. More info: https://github.com/pandas-dev/pandas/issues/30051

In [14]:
print((full_df == 5.397605346934028e-79).sum().sum())

1613351


In [15]:
full_df=full_df.replace(5.397605346934028e-79,0)

In [16]:
print((full_df == 5.397605346934028e-79).sum().sum())

0


In [17]:
print((full_df == 0).sum().sum())

1616733


6. Save full dataframe into a csv file and a pickle file

In [ ]:
# Save full dataframe into a csv file and a pickle file
full_df.to_csv('full_df.csv', index=False)
full_df.to_pickle('full_df.pkl')